# TensorMask
Xinlei Chen, Ross Girshick, Kaiming He, Piotr Dollár

[[`arXiv`](https://arxiv.org/abs/1903.12174)] [[`BibTeX`](#CitingTensorMask)]

For more information regarding this work please refer to the link below:
https://github.com/facebookresearch/detectron2/tree/main/projects/TensorMask

# Importing Necessary Libraries

In [ ]:
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

torch.cuda.empty_cache()

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [30, 15]

# import some common libraries
import numpy as np
import os, json, cv2, random
import re

import fiftyone as fo
from PIL import Image

In [ ]:
import logging


import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, hooks
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
logger = logging.getLogger("detectron2")

import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, launch
from detectron2.evaluation import COCOEvaluator, verify_results

from detectron2.evaluation import inference_on_dataset
from detectron2.data import build_detection_test_loader

from tensormask import add_tensormask_config

# Custom Function for Preparing Training Set 

In [ ]:
def make_seg_cotton_dicts(Train_data_path, image_id = 1):
    import io
    import ast
    dataset_list = []
    subset_folders = os.listdir(Train_data_path)

    for frames in subset_folders:
        if '.png' in frames:
            dict_holder = {}
            file_name = os.path.join(Train_data_path, frames)
            dict_holder["file_name"] = file_name
            dict_holder["height"], dict_holder["width"] = cv2.imread(file_name).shape[0:2]
            dict_holder["image_id"] = image_id
            dict_holder["fr_name"] = re.sub(r'\.png','',frames)
            #s = open(file_name[0:-4] + '.txt').read().replace(':','')
            annotations = []
            with open(file_name[0:-4] + '.txt') as folder:
                for (k,line) in enumerate(folder):
                    tmp = line.split('[')
                    segment = [ast.literal_eval('['+tmp[1])] # format = [[float]]
                    cat_n_bbox = tmp[0].split()
                    category = int(cat_n_bbox[0].replace(':', ''))
                    bbox = [float(cat_n_bbox[1]), float(cat_n_bbox[2]), float(cat_n_bbox[3]), float(cat_n_bbox[4])]
                    # dict_store has boxmode(0) = [x1,y1,x2,y2] not boxmode(1) = [x1,y1,w,h] as previous code (use code cautiously)
                    dict_annot = {
                        "bbox": bbox,
                        "bbox_mode": detectron2.structures.BoxMode(0),
                        "category_id": category,
                        "segmentation": segment
                    }
                    annotations.append(dict_annot)

                    

            dict_holder["annotations"] = annotations
            #bboxes = np.loadtxt(io.StringIO(s), usecols=(4,))
            
            if 'train' in Train_data_path:
                dataset_list.append(dict_holder)
                image_id += 1
            # what about the augmented images --> it does not append augmented images with this code? (else is valid for validation and test data)
            else:
                if 'aug' not in frames:
                    dataset_list.append(dict_holder)
                    image_id += 1
    
    return dataset_list

In [ ]:
Train_data_path = 'train_average'
Base_path = 'Cotton Fiber Project'
train_dataset_dicts = make_seg_cotton_dicts(Train_data_path)

In [ ]:
for d in ["train_average"]: #,,"val","test" (enter inside list for val data creation)
    DatasetCatalog.register("CFH_" + d,lambda d=d: make_seg_cotton_dicts(os.path.join(Base_path,d)))
    MetadataCatalog.get("CFH_" + d).thing_classes=["fiber"]

In [ ]:
metadata_train = MetadataCatalog.get("CFH_train_average")
metadata_train

If Training Set's COCO format already exists:

In [ ]:
register_coco_instances("CFH_train_average", {}, "CFH_train_average.json", "train_average")

# Train Model

## Function to Save the Detectron2 Config into Disk

In [ ]:
def cfg2yaml(cfg):
    
    with open(cfg.train.output_dir + "/Config.txt", 'w') as file:
        file.write(str(cfg))
    
    os.rename(cfg.train.output_dir + "/Config.txt", cfg.train.output_dir + "/Config.yaml")

In [ ]:
class Trainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, output_dir=output_folder)

## Setup Detectron2's Config

In [ ]:
args = {
    "config_file":"projects/TensorMask/configs/tensormask_R_50_FPN_6x.yaml"
}
"""
Create configs and perform basic setups.
"""
cfg = get_cfg()
add_tensormask_config(cfg)
cfg.merge_from_file(args["config_file"])
cfg.DATASETS.TRAIN = ("CFH_train_average",)
cfg.DATASETS.TEST = ("CFH_train_average",)
cfg.SOLVER.BASE_LR = 0.001  # Learning Rate
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.MAX_ITER = 50000
cfg.SOLVER.STEPS = (40000,45000)
cfg.MODEL.TENSOR_MASK.NUM_CLASSES = 1
cfg.OUTPUT_DIR = "Output"
# cfg.merge_from_list(args.opts)
cfg.freeze()
default_setup(cfg, args)

In [ ]:
cfg2yaml(cfg)
trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

### Display Training Loss

In [ ]:
plt.rcParams['figure.figsize'] = [14, 7]
def load_json_arr(json_path):
    lines = []
    with open(json_path, 'r') as f:
        for line in f:
            lines.append(json.loads(line))
    return lines

experiment_metrics = load_json_arr(cfg.OUTPUT_DIR + '/metrics.json')

# plt.legend(['training_loss', 'validation_loss'], loc='upper left')
plt.legend(['training_loss'])
plt.xlabel('Iteration')
plt.ylabel('Total Loss')
plt.savefig(cfg.OUTPUT_DIR +  '/Loss Curve.png')
plt.show()

# Evaluate Model

In [ ]:
cfg = get_cfg()
add_tensormask_config(cfg)
cfg.merge_from_file(args["config_file"])
cfg.DATASETS.TRAIN = ("CFH_train_average",)
cfg.DATASETS.TEST = ("CFH_train_average",)
cfg.SOLVER.IMS_PER_BATCH = 8
cfg.MODEL.TENSOR_MASK.NUM_CLASSES = 1
cfg.OUTPUT_DIR = "Output"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.TENSOR_MASK.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
# cfg.merge_from_list(args.opts)
cfg.freeze()
default_setup(cfg, args)

predictor = DefaultPredictor(cfg)

In [ ]:
evaluator = COCOEvaluator("CFH_train_average", output_dir=cfg.OUTPUT_DIR,use_fast_impl=False)
val_loader = build_detection_test_loader(cfg, "CFH_train_average")
print(inference_on_dataset(predictor.model, val_loader, evaluator))

## Visualize Model Output and Performance Using FiftyOne

In [ ]:
# dataset.delete()
dataset = fo.Dataset.from_dir(
    data_path= "Cotton Fiber Project/train_average",
    labels_path='Cotton Fiber Project/CFH_train_average.json',
    dataset_type=fo.types.COCODetectionDataset,
    label_types=["detections", "segmentations"],
    label_field = "ground_truth",
    #name="Model_2500_1024BatchSize_15LR"
)

In [ ]:
device = torch.device("cpu")

classes = ["fiber"]
torch.cuda.empty_cache()

# Add predictions to samples
with fo.ProgressBar() as pb:
    for sample in pb(dataset):
        # Load image
        image = cv2.imread(sample.filepath)
        im = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w , c = image.shape

        # Perform inference
        preds = predictor(image)
        if len(preds["instances"]) != 0:
            labels = preds["instances"].pred_classes.cpu().detach().numpy()
            scores = preds["instances"].scores.cpu().detach().numpy()
            masks = preds["instances"].pred_masks.cpu().detach().numpy()

            # Convert detections to FiftyOne format
            detections = []
            segmentations = []
            for label, score, seg in zip(labels, scores, masks):

                segmentations.append(
                    fo.Detection.from_mask(
                        mask=seg,
                        label=classes[label],
                        confidence=score
                    )
                )

            # Save predictions to dataset
            sample["predictions"] = fo.Detections(detections=segmentations)
            sample.save()

print("Finished adding predictions")

In [ ]:
results = predictions_view.evaluate_detections(
    "predictions",
    gt_field="ground_truth_segmentations",
    eval_key="eval",
    compute_mAP=True,
    use_masks=True,
    classes= classes,
    iou=0.5,
)

In [ ]:
results.print_report()

In [ ]:
session = fo.launch_app(dataset)

In [ ]:
# Export the dataset GTseg
dataset.export(
    labels_path= cfg.train.output_dir + "/GTsegmentation.json",
    dataset_type=fo.types.COCODetectionDataset,
    label_field = "ground_truth_segmentations",
)

# Export the dataset predictions
dataset.export(
    labels_path= cfg.train.output_dir + "/predictions.json" ,
    dataset_type=fo.types.COCODetectionDataset,
    label_field = "predictions",
)